In [29]:
import gymnasium as gym
import numpy as np
import torch
from utils import *
from dreamer import *
import pickle
torch.set_printoptions(threshold=2000, linewidth=200)
np.set_printoptions(threshold=2000, linewidth=200)

environmentName = "CarRacing-v3"
renderMode = None
numEpisodes = 1
stepCountLimit = 256

env = gym.make(environmentName, render_mode=renderMode)
observationShape = torch.tensor(env.observation_space.shape)
actionSize = torch.tensor(env.action_space.shape) if hasattr(env.action_space, 'shape') else np.array([env.action_space.n])
print(f"Env {environmentName} with observations {observationShape} and actions {actionSize}")
dreamer = Dreamer()

for episode in range(numEpisodes):
    observation, info = env.reset()
    observations, actions, rewards, dones = [observation[:,:,:]], [], [], []
    stepCount, totalReward, done = 1, 0, False
    while not done:
        stepCount += 1
        action = env.action_space.sample()*[1, 5, 0.2] # For now random action, scaled so we go a bit forward
        observation, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated or stepCount >= stepCountLimit
        totalReward += reward
        
        observations.append(observation[:,:,:]) # cutting out black strip
        actions.append(action)
        # We will get to those when world model will work
        # rewards.append(reward)
        # dones.append(done)
    print(f"Total Reward: {totalReward:.4f}")

    with open(f'episode_{episode}_data.pkl', 'wb') as f:
        obs = torch.from_numpy(np.transpose(np.stack(observations), (0, 3, 1, 2)))
        a = torch.from_numpy(np.stack(actions))
        print(f"Saving stacked observations of shape: {obs[0].shape}")
        pickle.dump({'observations': obs, 'actions': a}, f)
    # dreamer.train(observations, actions)#, rewards, dones)
env.close()

Env CarRacing-v3 with observations tensor([96, 96,  3]) and actions tensor([3])
Total Reward: 2.0862
Saving stacked observations of shape: torch.Size([3, 96, 96])


In [30]:
observation[:, :-12, :].shape

(96, 84, 3)